### This notebook is Final submission for the hackathon 

https://www.machinehack.com/hackathons/insurance_churn_prediction_weekend_hackathon_2/overview

Alert: This notebook is an old notebook so I couldn't structure the preprocessing and model code.

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 200)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

import keras
from keras.layers import LeakyReLU

Using TensorFlow backend.


In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train['labels'].value_counts()

0    29941
1     3967
Name: labels, dtype: int64

In [4]:
pd.read_excel('sample_submission.xlsx').head(5)

,labels
0,1
1,1
2,1
3,1
4,1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33908 entries, 0 to 33907
Data columns (total 17 columns):
feature_0     33908 non-null float64
feature_1     33908 non-null float64
feature_2     33908 non-null float64
feature_3     33908 non-null float64
feature_4     33908 non-null float64
feature_5     33908 non-null float64
feature_6     33908 non-null float64
feature_7     33908 non-null int64
feature_8     33908 non-null int64
feature_9     33908 non-null int64
feature_10    33908 non-null int64
feature_11    33908 non-null int64
feature_12    33908 non-null int64
feature_13    33908 non-null int64
feature_14    33908 non-null int64
feature_15    33908 non-null int64
labels        33908 non-null int64
dtypes: float64(7), int64(10)
memory usage: 4.4 MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11303 entries, 0 to 11302
Data columns (total 16 columns):
feature_0     11303 non-null float64
feature_1     11303 non-null float64
feature_2     11303 non-null float64
feature_3     11303 non-null float64
feature_4     11303 non-null float64
feature_5     11303 non-null float64
feature_6     11303 non-null float64
feature_7     11303 non-null int64
feature_8     11303 non-null int64
feature_9     11303 non-null int64
feature_10    11303 non-null int64
feature_11    11303 non-null int64
feature_12    11303 non-null int64
feature_13    11303 non-null int64
feature_14    11303 non-null int64
feature_15    11303 non-null int64
dtypes: float64(7), int64(9)
memory usage: 1.4 MB


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('labels', axis=1), train['labels'],
                                                    test_size=0.30, stratify=train['labels'])

In [8]:
import tensorflow as tf
from keras.losses import binary_crossentropy, categorical_crossentropy
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)


In [9]:
leaky_relu_alpha =0.02
model_mlp = Sequential()
model_mlp.add(Dense(60, input_dim=X_train.shape[1], activation='relu'))
# model_mlp.add(LeakyReLU(alpha=leaky_relu_alpha))
model_mlp.add(Dense(35, kernel_initializer='normal', activation='relu'))
# model_mlp.add(LeakyReLU(alpha=leaky_relu_alpha))
model_mlp.add(Dense(15, kernel_initializer='normal', activation='relu'))
# model_mlp.add(LeakyReLU(alpha=leaky_relu_alpha))
model_mlp.add(Dense(1, activation='sigmoid'))
model_mlp.compile(optimizer='adam', loss=f1_loss, metrics=['accuracy', f1])
model_mlp.summary()

W0425 14:09:45.023117 4501493184 deprecation.py:323] From <ipython-input-8-32aab2f104de>:16: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 35)                2135      
_________________________________________________________________
dense_3 (Dense)              (None, 15)                540       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 16        
Total params: 3,711
Trainable params: 3,711
Non-trainable params: 0
_________________________________________________________________


In [12]:
callback = [keras.callbacks.EarlyStopping(monitor='val_f1', min_delta=0, patience=10,
                                        mode='auto', restore_best_weights=True),
          keras.callbacks.ReduceLROnPlateau(monitor='val_f1', factor=0.25, patience=6,
                                            min_lr=0.00001)]

In [13]:
mlp_history = model_mlp.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            epochs=150, verbose=1, batch_size=128, callbacks=callback)

Train on 23735 samples, validate on 10173 samples
Epoch 1/150
23735/23735 [==============================] - 0s 14us/step - loss: 0.4982 - accuracy: 0.8565 - f1: 0.5081 - val_loss: 0.4961 - val_accuracy: 0.8680 - val_f1: 0.5116
Epoch 2/150
23735/23735 [==============================] - 0s 13us/step - loss: 0.4826 - accuracy: 0.8670 - f1: 0.5213 - val_loss: 0.5124 - val_accuracy: 0.8779 - val_f1: 0.4864
Epoch 3/150
23735/23735 [==============================] - 0s 13us/step - loss: 0.4739 - accuracy: 0.8693 - f1: 0.5319 - val_loss: 0.4797 - val_accuracy: 0.8748 - val_f1: 0.5245
Epoch 4/150
23735/23735 [==============================] - 0s 13us/step - loss: 0.4688 - accuracy: 0.8710 - f1: 0.5354 - val_loss: 0.4783 - val_accuracy: 0.8572 - val_f1: 0.5253
Epoch 5/150
23735/23735 [==============================] - 0s 14us/step - loss: 0.4664 - accuracy: 0.8716 - f1: 0.5347 - val_loss: 0.4810 - val_accuracy: 0.8734 - val_f1: 0.5236
Epoch 6/150
23735/23735 [==============================] - 0

In [42]:
mlp_history = model_mlp.fit(train.drop('labels', axis=1), train['labels'],
                            validation_data=(X_test, y_test),
                            epochs=150, verbose=1, batch_size=128)

Train on 33908 samples, validate on 10173 samples
Epoch 1/150
33908/33908 [==============================] - 0s 15us/step - loss: 0.4400 - accuracy: 0.8806 - f1: 0.5635 - val_loss: 0.4413 - val_accuracy: 0.8700 - val_f1: 0.5612
Epoch 2/150
33908/33908 [==============================] - 0s 13us/step - loss: 0.4329 - accuracy: 0.8822 - f1: 0.5695 - val_loss: 0.4328 - val_accuracy: 0.8787 - val_f1: 0.5696
Epoch 3/150
33908/33908 [==============================] - 0s 13us/step - loss: 0.4341 - accuracy: 0.8846 - f1: 0.5666 - val_loss: 0.4259 - val_accuracy: 0.8839 - val_f1: 0.5769
Epoch 4/150
33908/33908 [==============================] - 0s 13us/step - loss: 0.4270 - accuracy: 0.8857 - f1: 0.5746 - val_loss: 0.4401 - val_accuracy: 0.8750 - val_f1: 0.5612
Epoch 5/150
33908/33908 [==============================] - 0s 13us/step - loss: 0.4285 - accuracy: 0.8854 - f1: 0.5731 - val_loss: 0.4355 - val_accuracy: 0.8734 - val_f1: 0.5667
Epoch 6/150
33908/33908 [==============================] - 0

Epoch 92/150
33908/33908 [==============================] - 0s 11us/step - loss: 0.3957 - accuracy: 0.8967 - f1: 0.6044 - val_loss: 0.3877 - val_accuracy: 0.8977 - val_f1: 0.6125
Epoch 93/150
33908/33908 [==============================] - 0s 11us/step - loss: 0.3908 - accuracy: 0.8980 - f1: 0.6094 - val_loss: 0.3875 - val_accuracy: 0.9019 - val_f1: 0.6164
Epoch 94/150
33908/33908 [==============================] - 0s 12us/step - loss: 0.3930 - accuracy: 0.8984 - f1: 0.6086 - val_loss: 0.3897 - val_accuracy: 0.8988 - val_f1: 0.6092
Epoch 95/150
33908/33908 [==============================] - 0s 11us/step - loss: 0.3916 - accuracy: 0.8981 - f1: 0.6090 - val_loss: 0.4024 - val_accuracy: 0.8882 - val_f1: 0.5978
Epoch 96/150
33908/33908 [==============================] - 0s 11us/step - loss: 0.3882 - accuracy: 0.8980 - f1: 0.6120 - val_loss: 0.3887 - val_accuracy: 0.9013 - val_f1: 0.6111
Epoch 97/150
33908/33908 [==============================] - 0s 11us/step - loss: 0.3967 - accuracy: 0.896

In [44]:
model_mlp.predict_classes(test).flatten()

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [45]:
pd.DataFrame({'labels':model_mlp.predict_classes(test).flatten()
             })['labels'].value_counts()

0    9766
1    1537
Name: labels, dtype: int64

In [46]:
pd.DataFrame({
    'labels':model_mlp.predict_classes(test).flatten()
             }).to_excel('Submission_3.xlsx', index=False)